In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
#import util
# pd.set_option('display.float_format', lambda x: '%.3f' % x)

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from collections import OrderedDict
from time import time
import os

from scipy.optimize import fmin_powell
from scipy import integrate

import theano as thno
import theano.tensor as T
import arviz as az

az.style.use('arviz-darkgrid')

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin

class AdditionalAttributes(BaseEstimator, TransformerMixin):
    def __init__(self, include_additional_attr = True): # no *args or **kargs
        self.include_additional_attr = include_additional_attr
        self.guille_feature_names = ['H', 'dest_A_1', 'dest_A_2', 'dest_A_3', 'dest_A_4', 'dest_A_5',
                                     'dest_A_6', 'dest_I', 'dest_dTR', 'dest_hK', 'dest_hM', 'dest_mR', 
                                     'src_A_1', 'src_A_2', 'src_A_3', 'src_A_4', 'src_A_5', 'src_A_6', 
                                     'src_I', 'src_dTR', 'src_hK', 'src_hM','src_mR']
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        if self.include_additional_attr:
            return X # might need to check if this returns pandas or a numpy, i need a numpy
        else:
            return X[self.guille_feature_names]

In [3]:
from joblib import dump

def save_models(saveas, rnd_estimator, lr_estimator):
    save_to = '../models/'
    saveas = os.path.join(save_to, saveas)
    if os.path.exists(saveas):
        raise FileExistsError(f'The path "{os.path.abspath(saveas)}" already exists, ' 
                              'please choose a new name.')
    else:
        rfc_dir = os.path.join(saveas, 'rfc')
        bayes_dir = os.path.join(saveas, 'bayes/')
        
        os.makedirs(rfc_dir)
        os.mkdir(bayes_dir)
    
    # save random forest estimator
    dump(rnd_estimator, os.path.join(rfc_dir, 'model.rnd'))
    
    # save bayesian logistic regression model
    lr_estimator.save(bayes_dir)

Set Project Parameters

In [4]:
include_additional_attr = False
dataset_filepath = "../data/processed/small-network-5/dataset.h5"
dataset_key = 'TVNWiGXGlrOnAwm'

if not os.path.exists(dataset_filepath):
    raise FileNotFoundError(f'{os.path.abspath(dataset_filepath)} does not exist.')

# Explore Dataset

In [5]:
# dataset = pd.read_csv(dataset_filepath)
dataset = pd.read_hdf(dataset_filepath, key=dataset_key)

# Remove user_ids from dataframe
dataset = dataset.drop(['src_user_id', 'dest_user_id'], axis=1)
dataset.head()

,H,dest_A_1,dest_A_2,dest_A_3,dest_A_4,dest_A_5,dest_A_6,dest_I,dest_avg_negative_sentiment_of_tweets,dest_avg_number_followers,...,src_ratio_of_retweets_to_tweets,src_ratio_of_tweet_per_time_period_1,src_ratio_of_tweet_per_time_period_2,src_ratio_of_tweet_per_time_period_3,src_ratio_of_tweet_per_time_period_4,src_ratio_of_tweets_that_got_retweeted_per_time_period_1,src_ratio_of_tweets_that_got_retweeted_per_time_period_2,src_ratio_of_tweets_that_got_retweeted_per_time_period_3,src_ratio_of_tweets_that_got_retweeted_per_time_period_4,y
0,0.003559,0.188514,0.015605,0.071473,0.276841,0.280587,0.166979,1.000000,1.0,1.000000,...,0.781008,0.289514,0.274667,0.246520,0.189298,0.012682,0.003402,0.021961,0.013610,0
1,0.000000,0.296296,0.037037,0.000000,0.222222,0.148148,0.296296,0.037007,1.0,0.026874,...,0.781008,0.289514,0.274667,0.246520,0.189298,0.012682,0.003402,0.021961,0.013610,0
2,0.010724,0.223320,0.166008,0.037549,0.033597,0.221344,0.318182,0.693531,1.0,0.322489,...,0.996608,0.216775,0.048104,0.467777,0.267345,0.012951,0.003084,0.026210,0.021277,0
3,0.000756,0.195604,0.065934,0.000000,0.105495,0.393407,0.239560,0.623629,1.0,0.309760,...,0.815960,0.104856,0.018868,0.497062,0.379214,0.048252,0.008661,0.177235,0.129292,1
4,0.000000,0.056924,0.015030,0.010233,0.453790,0.285577,0.178446,1.000000,1.0,1.000000,...,0.954813,0.233055,0.007119,0.260600,0.499226,0.217270,0.005881,0.231198,0.422779,1


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 84 columns):
H                                                            5 non-null float64
dest_A_1                                                     5 non-null float64
dest_A_2                                                     5 non-null float64
dest_A_3                                                     5 non-null float64
dest_A_4                                                     5 non-null float64
dest_A_5                                                     5 non-null float64
dest_A_6                                                     5 non-null float64
dest_I                                                       5 non-null float64
dest_avg_negative_sentiment_of_tweets                        5 non-null float64
dest_avg_number_followers                                    5 non-null float64
dest_avg_number_friends                                      5 non-null float64
dest_avg_number_of_

In [7]:
dataset.describe()

,H,dest_A_1,dest_A_2,dest_A_3,dest_A_4,dest_A_5,dest_A_6,dest_I,dest_avg_negative_sentiment_of_tweets,dest_avg_number_followers,...,src_ratio_of_retweets_to_tweets,src_ratio_of_tweet_per_time_period_1,src_ratio_of_tweet_per_time_period_2,src_ratio_of_tweet_per_time_period_3,src_ratio_of_tweet_per_time_period_4,src_ratio_of_tweets_that_got_retweeted_per_time_period_1,src_ratio_of_tweets_that_got_retweeted_per_time_period_2,src_ratio_of_tweets_that_got_retweeted_per_time_period_3,src_ratio_of_tweets_that_got_retweeted_per_time_period_4,y
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.0,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.003008,0.192132,0.059923,0.023851,0.218389,0.265813,0.239893,0.670833,1.0,0.531825,...,0.865880,0.226743,0.124685,0.343696,0.304876,0.060767,0.004886,0.095713,0.120113,0.400000
std,0.004555,0.086789,0.062836,0.030736,0.162587,0.090432,0.067829,0.394075,0.0,0.443420,...,0.102344,0.075624,0.137726,0.127189,0.133626,0.088827,0.002392,0.100885,0.176172,0.547723
min,0.000000,0.056924,0.015030,0.000000,0.033597,0.148148,0.166979,0.037007,1.0,0.026874,...,0.781008,0.104856,0.007119,0.246520,0.189298,0.012682,0.003084,0.021961,0.013610,0.000000
25%,0.000000,0.188514,0.015605,0.000000,0.105495,0.221344,0.178446,0.623629,1.0,0.309760,...,0.781008,0.216775,0.018868,0.246520,0.189298,0.012682,0.003402,0.021961,0.013610,0.000000
50%,0.000756,0.195604,0.037037,0.010233,0.222222,0.280587,0.239560,0.693531,1.0,0.322489,...,0.815960,0.233055,0.048104,0.260600,0.267345,0.012951,0.003402,0.026210,0.021277,0.000000
75%,0.003559,0.223320,0.065934,0.037549,0.276841,0.285577,0.296296,1.000000,1.0,1.000000,...,0.954813,0.289514,0.274667,0.467777,0.379214,0.048252,0.005881,0.177235,0.129292,1.000000
max,0.010724,0.296296,0.166008,0.071473,0.453790,0.393407,0.318182,1.000000,1.0,1.000000,...,0.996608,0.289514,0.274667,0.497062,0.499226,0.217270,0.008661,0.231198,0.422779,1.000000


In [ ]:
sns.stripplot(x="y", y="H", data=dataset, jitter=True)

In [8]:
X = dataset.drop("y", axis=1) # drop labels for training set
y = dataset["y"].copy()

feature_names = X.columns.values

y.value_counts()

0    3
1    2
Name: y, dtype: int64

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

pipeline = Pipeline([
    ('attribs_adder', AdditionalAttributes(include_additional_attr = include_additional_attr)),
    ('imputer', SimpleImputer(strategy="mean")),
])

# ('std_scaler', StandardScaler()),

X = pipeline.fit_transform(X)

In [14]:
# resample using SMOTE balance
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=42).fit_resample(X, y)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 6

In [ ]:
# column values based on whether additional attributes is true or false
# include_additional_attr = pipeline.named_steps['attribs_adder'].include_additional_attr
guille_feature_names = pipeline.named_steps['attribs_adder'].guille_feature_names

if not include_additional_attr:
    feature_names = guille_feature_names

In [ ]:
X_resampled = pd.DataFrame(X_resampled, columns=feature_names)
y_resampled = pd.Series(y_resampled)

In [ ]:
y_resampled.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [ ]:
print ('x_train shape: ', X_train.shape)
print ('y_train shape: ', y_train.shape)
print ('x_test shape: ', X_test.shape)
print ('y_test shape: ', y_test.shape)

## Exploring the data

In [ ]:
#g = seaborn.pairplot(data)

In [ ]:
# Compute the correlation matrix
corr = X_train.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)



## Build Bayesian Logistics Regression Model

In [ ]:
X = X_resampled[features]
y = y_resampled.copy()

In [ ]:
y.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print ('x_train shape: ', X_train.shape)
print ('y_train shape: ', y_train.shape)
print ('x_test shape: ', X_test.shape)
print ('y_test shape: ', y_test.shape)

In [ ]:
#g = seaborn.pairplot(data)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
from pymc3_models.models.LogisticRegression import LogisticRegression

LR = LogisticRegression()
LR.fit(X_train, y_train, inference_type='nuts', inference_args={'draws': 2000})

In [ ]:
az.plot_trace(LR.trace)

In [ ]:
LR.summary

In [ ]:
y_pred = LR.predict(X_test)

In [ ]:
y_score = LR.predict_proba(X_test)

### Performance

In [ ]:
LR.score(X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['0', '1']); ax.yaxis.set_ticklabels(['1', '0']);

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, y_score)

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, threshold = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc_score(y_test, y_score))
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
save_models(saveas='big-network-full-attr', rnd_estimator=clf, lr_estimator=LR)